<a href="https://colab.research.google.com/github/edtorrente/Deep-Learning-Charity-Funding-Predictor/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#https://drive.google.com/file/d/11zI0VBjhp2ZYkSc42SIGe1gsrHpXa8WC/view?usp=sharing

fileDownload = drive.CreateFile({'id':'11zI0VBjhp2ZYkSc42SIGe1gsrHpXa8WC'})

In [4]:
fileDownload.GetContentFile("charity_data.csv")

# **Attempt1: Dropping more or fewer columns.**


In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

application_df= pd.read_csv('charity_data.csv', delimiter=',')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Include 'USE_CASE' to the original dropped columns of 'EIN' and 'NAME'

application_df.drop(['EIN', 'NAME', 'USE_CASE'], axis=1, inplace=True)

In [7]:
# What are the unique values in each column.

application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
# need to bin the 'APPLICATION_TYPE' column

application_column = application_df['APPLICATION_TYPE'].value_counts()
application_column.head()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace =application_column[application_column < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_column = application_df['CLASSIFICATION'].value_counts()
classification_column

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE

application_df['CLASSIFICATION'].value_counts() > 1

C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C4120    False
C8210    False
C2561    False
C4500    False
C2150    False
Name: CLASSIFICATION, Length: 71, dtype: bool

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace =classification_column[classification_column <800].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
y = application_df['IS_SUCCESSFUL'].values
y

array([1, 1, 0, ..., 0, 1, 0])

In [15]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
X

array([[1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 1.0859000e+05, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 3.6500179e+07, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]])

In [16]:
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**Compile, Train and Evaluate the Model**

---



In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_features = len(X_train_scaled[0])
hidden_node_layer1 = 80
hidden_node_layer2 = 30
hidden_node_layer3 = 1
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_node_layer1, input_dim= input_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_node_layer2,  activation='relu'))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1,  activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3120      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,581
Trainable params: 5,581
Non-trainable params: 0
_________________________________________________________________


In [19]:
# import dependencies for callbacks

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path = 'checkpoints/weights.{epoch:02d}.hdf5' 

In [20]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
from re import VERBOSE
# Create a callback that saves the model's weights every 5 epochs.

cpp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000
)

In [22]:
# Train the model
#  YOUR CODE GOES HERE
neural_train = nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 5s 5ms/step - loss: 67038.1172 - accuracy: 0.4892
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 27471.7285 - accuracy: 0.5112
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 47503.2266 - accuracy: 0.4958
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 5479.4995 - accuracy: 0.4937
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 18539.5137 - accuracy: 0.4899
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 49170.1055 - accuracy: 0.5029
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 20236.9824 - accuracy: 0.4890
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 33755.3398 - accuracy: 0.4972
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 21579.6992 - accuracy: 0.4878
Epoch 10/50
804/804 [==============================] - 2s 2ms/ste

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7179 - accuracy: 0.5738 - 473ms/epoch - 2ms/step
Loss: 0.7179168462753296, Accuracy: 0.5737609267234802


In [24]:
#export model in HDF5 file

from google.colab import files

nn.save('/content/attempt1.h5')
files.download('/content/attempt1.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Attempt2: Add neurons to hidden layers and add more layers.**

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_features = len(X_train_scaled[0])
hidden_node_layer1 = 100
hidden_node_layer2 = 50
hidden_node_layer3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_node_layer1, input_dim= input_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_node_layer2,  activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node_layer3,  activation='relu'))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1,  activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               3900      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 10)                510       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,471
Trainable params: 9,471
Non-trainable params: 0
_________________________________________________________________


In [26]:
# import dependencies for callbacks

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path = 'checkpoints/weights.{epoch:02d}.hdf5' 

In [27]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
from re import VERBOSE
# Create a callback that saves the model's weights every 5 epochs.

cpp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000
)

In [29]:
# Train the model
#  YOUR CODE GOES HERE
neural_train = nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 23968.1250 - accuracy: 0.4946
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 15953.0078 - accuracy: 0.5241
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.6921 - accuracy: 0.5318
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5318
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6878 - accuracy: 0.5343 - 489ms/epoch - 2ms/step
Loss: 0.6878265142440796, Accuracy: 0.5343440175056458


In [31]:
#export model in HDF5 file

from google.colab import files

nn.save('/content/attempt2.h5')
files.download('/content/attempt2.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Attempt3: Change activation layer at the output to "tanh".**

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_features = len(X_train_scaled[0])
hidden_node_layer1 = 100
hidden_node_layer2 = 50
hidden_node_layer3 = 20
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_node_layer1, input_dim= input_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_node_layer2,  activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node_layer3,  activation='relu'))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1,  activation='tanh'))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               3900      
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 20)                1020      
                                                                 
 dense_10 (Dense)            (None, 1)                 21        
                                                                 
Total params: 9,991
Trainable params: 9,991
Non-trainable params: 0
_________________________________________________________________


In [33]:
# import dependencies for callbacks

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path = 'checkpoints/weights.{epoch:02d}.hdf5' 

In [34]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
from re import VERBOSE
# Create a callback that saves the model's weights every 5 epochs.

cpp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000
)

In [36]:
# Train the model
#  YOUR CODE GOES HERE
neural_train = nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 8.1920 - accuracy: 0.4686
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 7.9449 - accuracy: 0.4660 - 486ms/epoch - 2ms/step
Loss: 7.944907188415527, Accuracy: 0.4660058319568634


In [38]:
#export model in HDF5 file

from google.colab import files

nn.save('/content/attempt3.h5')
files.download('/content/attempt3.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>